In [1]:
import datetime
import stock_marker_binance
import model_design
import pandas as pd
from sklearn.utils import class_weight
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
EPOCHS =  2000
TIME_FRAME = [2, 3, 21]
TRAIN_FRACTION = 0.75

In [3]:
def dataframe_to_tensor(df, timestep_length: int):
    
    '''
    Converts a pandas dataframe to a tensor
    
    Input: Pandas dataframe
            timestep_length (int) - the length of the timestep
    Output: Tensor
    
    '''
    
    # Convert the dataframe to a numpy array
    df = np.array(df)

    n_seq = len(df) - timestep_length + 1
    return np.array([df[i:(i+timestep_length)] for i in range(n_seq)])

In [4]:
def get_train_test_sets(data, train_frac):
    n_train = int(data.shape[0] * train_frac)
    x_train = data[:n_train, :, :-4]
    y_train = data[:n_train, -1:, -4:].reshape(-1,4)
    x_test = data[n_train:, :, :-4]
    y_test = data[n_train:, -1:, -4:].reshape(-1,4)
    return x_train, y_train, x_test, y_test

In [5]:
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [6]:
df = pd.read_csv('ETH_1h_history as of 09-01-2024.csv', index_col=0)

In [7]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1577840399999,1.000930e+06,2504
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1577843999999,1.474278e+06,4885
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1577847599999,9.940256e+05,3046
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1577851199999,6.473610e+05,2818
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1577854799999,4.430067e+05,2264


In [8]:
df_result = stock_marker_binance.sorting_timestamp(df)

df_result['hour_sin'] = np.sin(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)
df_result['hour_cos'] = np.cos(df_result['timestamp'].dt.hour / 24 * 2 * np.pi)

df_result['day_sin'] = np.sin(df_result['timestamp'].dt.day / 30 * 2 * np.pi)
df_result['day_cos'] = np.cos(df_result['timestamp'].dt.day / 30 * 2 * np.pi)

df_result['mon_sin'] = np.sin(df_result['timestamp'].dt.month / 12 * 2 * np.pi)
df_result['mon_cos'] = np.cos(df_result['timestamp'].dt.month / 12 * 2 * np.pi)

df_result['weekday_sin'] = np.sin(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)
df_result['weekday_cos'] = np.cos(df_result['timestamp'].dt.weekday / 7 * 2 * np.pi)

df_result['year'] = df_result['timestamp'].dt.year


In [9]:
df_result.head(10)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year
0,2020-01-01 00:00:00,129.16,129.19,128.68,128.87,7769.17336,1577840399999,1.000930e+06,2504,0.000000,1.000000e+00,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
1,2020-01-01 01:00:00,128.87,130.65,128.78,130.64,11344.65516,1577843999999,1.474278e+06,4885,0.258819,9.659258e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
2,2020-01-01 02:00:00,130.63,130.98,130.35,130.85,7603.35623,1577847599999,9.940256e+05,3046,0.500000,8.660254e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
3,2020-01-01 03:00:00,130.85,130.89,129.94,130.20,4968.55433,1577851199999,6.473610e+05,2818,0.707107,7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
4,2020-01-01 04:00:00,130.21,130.74,130.15,130.20,3397.90747,1577854799999,4.430067e+05,2264,0.866025,5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
5,2020-01-01 05:00:00,130.20,130.47,130.11,130.30,4243.60640,1577858399999,5.529727e+05,2426,0.965926,2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
6,2020-01-01 06:00:00,130.31,130.75,130.26,130.44,3668.90166,1577861999999,4.789442e+05,2347,1.000000,6.123234e-17,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
7,2020-01-01 07:00:00,130.47,130.71,130.14,130.24,4147.17413,1577865599999,5.407707e+05,2568,0.965926,-2.588190e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
8,2020-01-01 08:00:00,130.24,130.41,129.87,130.36,7541.44497,1577869199999,9.808940e+05,3039,0.866025,-5.000000e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020
9,2020-01-01 09:00:00,130.40,130.62,130.13,130.17,4808.20496,1577872799999,6.268190e+05,2696,0.707107,-7.071068e-01,0.207912,0.978148,0.5,0.866025,0.974928,-0.222521,2020


In [10]:

# df_result = stock_marker_binance.adding_MAs(df_result, 'close')
# df_result = stock_marker_binance.adding_MAs(df_result, 'quote_asset_volume')
# df_result = stock_marker_binance.adding_MAs(df_result, 'num_trades')
# print("Complete adding MAs")
# df_result = stock_marker_binance.adding_ratio(df_result, 'close')
# df_result = stock_marker_binance.adding_ratio(df_result, 'quote_asset_volume')
# df_result = stock_marker_binance.adding_ratio(df_result, 'num_trades')
# print("Complete adding ratios")
df_result = stock_marker_binance.marking_buy_sell_actions(df_result)
print("Complete marking buy sell actions")
df_result = stock_marker_binance.marking_adjacent_actions(df_result)
print("Complete marking adjacent actions")
df_result = stock_marker_binance.marking_hold_wait_actions(df_result)
print("Complete marking hold wait actions")

df_result = df_result.drop(columns=['timestamp','open', 'high', 'low' ,'close_time', 'volume',
                                    # 'close',  'quote_asset_volume', 'num_trades'
                                    ])
df_result=df_result.dropna()
df_result.reset_index(inplace=True, drop=True)
# df_result = stock_marker_binance.renaming_cols(df_result, STOCK_TICKER + "_", ['timestamp', 'action'])

Complete adding MAs
Complete adding ratios
Complete marking buy sell actions
Complete marking adjacent actions
Complete marking hold wait actions


In [11]:
df_result.groupby('action').count()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,MA3_close,...,RATIO_quote_asset_volume_and_MA192,RATIO_quote_asset_volume_and_MA384,RATIO_num_trades_and_MA3,RATIO_num_trades_and_MA6,RATIO_num_trades_and_MA12,RATIO_num_trades_and_MA24,RATIO_num_trades_and_MA48,RATIO_num_trades_and_MA96,RATIO_num_trades_and_MA192,RATIO_num_trades_and_MA384
action,,,,,,,,,,,,,,,,,,,,,
B,2341,2341,2341,2341,2341,2341,2341,2341,2341,2341,...,2341,2341,2341,2341,2341,2341,2341,2341,2341,2341
H,11122,11122,11122,11122,11122,11122,11122,11122,11122,11122,...,11122,11122,11122,11122,11122,11122,11122,11122,11122,11122
S,2650,2650,2650,2650,2650,2650,2650,2650,2650,2650,...,2650,2650,2650,2650,2650,2650,2650,2650,2650,2650
W,18832,18832,18832,18832,18832,18832,18832,18832,18832,18832,...,18832,18832,18832,18832,18832,18832,18832,18832,18832,18832


In [12]:
# # Dropping cols with MAs
# MA_list =[]
# for col in df_result.columns:
#     if col[:2]=="MA":
#         MA_list.append(col)
# df_result.drop(columns=MA_list, inplace=True)

In [13]:
df_result

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,MA3_close,...,RATIO_quote_asset_volume_and_MA384,RATIO_num_trades_and_MA3,RATIO_num_trades_and_MA6,RATIO_num_trades_and_MA12,RATIO_num_trades_and_MA24,RATIO_num_trades_and_MA48,RATIO_num_trades_and_MA96,RATIO_num_trades_and_MA192,RATIO_num_trades_and_MA384,action
0,-0.258819,9.659258e-01,-0.207912,-0.978148,0.5,0.866025,0.433884,-0.900969,2020,163.913333,...,0.690739,0.850917,0.873336,0.648463,0.646269,0.538017,0.542010,0.642564,0.722227,H
1,0.000000,1.000000e+00,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,164.043333,...,1.548873,1.503096,1.473624,1.252260,1.221725,1.053068,1.006492,1.194764,1.341485,B
2,0.258819,9.659258e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,164.283333,...,0.771169,0.834667,0.848712,0.775119,0.765130,0.641521,0.594868,0.708056,0.794659,S
3,0.500000,8.660254e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,164.770000,...,1.904065,1.273192,1.511792,1.481664,1.482436,1.264117,1.174457,1.396465,1.570252,W
4,0.707107,7.071068e-01,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,165.223333,...,2.078477,1.272368,1.541692,1.554804,1.599799,1.401618,1.292669,1.540500,1.734046,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34940,0.707107,7.071068e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,2310.196667,...,0.577831,0.811817,0.644617,0.455961,0.478128,0.582740,0.692765,0.591776,0.602909,W
34941,0.866025,5.000000e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,2312.143333,...,0.537459,1.046072,0.677817,0.500209,0.510001,0.617640,0.738253,0.630463,0.642304,W
34942,0.965926,2.588190e-01,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,2311.033333,...,0.562553,1.000576,0.682097,0.522844,0.495423,0.596806,0.715863,0.611617,0.624796,W
34943,1.000000,6.123234e-17,0.951057,-0.309017,0.5,0.866025,0.781831,0.623490,2024,2308.656667,...,0.532272,0.998771,0.918378,0.640296,0.516737,0.602169,0.727579,0.619264,0.633773,W


In [14]:
# le = sklearn_prep.LabelEncoder()
# df_result['action'] = le.fit_transform(df_result['action'])

enc = OneHotEncoder()

OHE_array = enc.fit_transform(df_result[['action']]).toarray()

class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                 classes=np.unique(df_result['action']),
                                                 y=df_result['action'])



class_weights_dict = dict(zip(enc.categories_[0], class_weights))

print(class_weights)
print(class_weights_dict.keys())


[3.73184537 0.78549272 3.29669811 0.46390452]
dict_keys(['B', 'H', 'S', 'W'])


In [15]:
class_weights_dict=dict(zip(range(0,4), class_weights))

In [16]:
class_weights_dict

{0: 3.731845365228535,
 1: 0.7854927171372056,
 2: 3.296698113207547,
 3: 0.46390452421410366}

In [17]:
OHE_df = pd.DataFrame(OHE_array, columns=enc.categories_[0])

In [18]:
df_result = pd.merge(df_result, OHE_df, left_index=True, right_index=True)
df_result.drop(columns=['action'], inplace=True)

In [19]:
print(class_weights_dict)

{0: 3.731845365228535, 1: 0.7854927171372056, 2: 3.296698113207547, 3: 0.46390452421410366}


In [20]:
df_result.head()

,hour_sin,hour_cos,day_sin,day_cos,mon_sin,mon_cos,weekday_sin,weekday_cos,year,MA3_close,...,RATIO_num_trades_and_MA12,RATIO_num_trades_and_MA24,RATIO_num_trades_and_MA48,RATIO_num_trades_and_MA96,RATIO_num_trades_and_MA192,RATIO_num_trades_and_MA384,B,H,S,W
0,-0.258819,0.965926,-0.207912,-0.978148,0.5,0.866025,0.433884,-0.900969,2020,163.913333,...,0.648463,0.646269,0.538017,0.542010,0.642564,0.722227,0.0,1.0,0.0,0.0
1,0.000000,1.000000,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,164.043333,...,1.252260,1.221725,1.053068,1.006492,1.194764,1.341485,1.0,0.0,0.0,0.0
2,0.258819,0.965926,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,164.283333,...,0.775119,0.765130,0.641521,0.594868,0.708056,0.794659,0.0,0.0,1.0,0.0
3,0.500000,0.866025,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,164.770000,...,1.481664,1.482436,1.264117,1.174457,1.396465,1.570252,0.0,0.0,0.0,1.0
4,0.707107,0.707107,-0.406737,-0.913545,0.5,0.866025,-0.433884,-0.900969,2020,165.223333,...,1.554804,1.599799,1.401618,1.292669,1.540500,1.734046,0.0,0.0,0.0,1.0


In [21]:
max_val_PR = 0

for time_frame in TIME_FRAME:
    df_tensor = dataframe_to_tensor(df_result, time_frame)
    x_train, y_train, x_test, y_test = get_train_test_sets(df_tensor, TRAIN_FRACTION)
    
    df_tensor.shape

    print(x_train.shape)
    print(y_train.shape)
    print(x_test.shape)
    print(y_test.shape)
    model, history = model_design.model_design(EPOCHS, x_train, y_train, x_test, y_test, class_weights_dict)

    val_PR = round(max(history.history['val_PR']),3)
    
    x = datetime.datetime.now()
    
    model_name = "models/" + x.strftime("%d-%m-%Y %H-%M-%S") +" val_PR " + str(val_PR) + " time frame " + str(time_frame) + ".hd5"

    if val_PR > max_val_PR:
        max_val_PR = val_PR
        model.save(model_name,  save_format = 'h5')
    
    print(model_name)
    print(val_PR)
    print(150 * "-")


(26208, 2, 57)
(26208, 4)
(8736, 2, 57)
(8736, 4)



C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


models/20-01-2024 22-28-19 val_PR 0.656 time frame 2.hd5
0.656
------------------------------------------------------------------------------------------------------------------------------------------------------
(26207, 3, 57)
(26207, 4)
(8736, 3, 57)
(8736, 4)
models/20-01-2024 22-30-34 val_PR 0.637 time frame 3.hd5
0.637
------------------------------------------------------------------------------------------------------------------------------------------------------
(26193, 21, 57)
(26193, 4)
(8732, 21, 57)
(8732, 4)
models/20-01-2024 22-35-32 val_PR 0.589 time frame 21.hd5
0.589
------------------------------------------------------------------------------------------------------------------------------------------------------
